In [1]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import Lipinski, Descriptors, rdMolDescriptors, AllChem, PandasTools
from pandarallel import pandarallel
import numpy as np
pandarallel.initialize(progress_bar=False)
import tmap as tm
from map4 import MAP4Calculator
import os
import joblib
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
folder = "/data/coconut/"

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
coconut = pd.read_pickle(folder + "coconut_prop.pkl")

In [3]:
origins = ["plants", "fungi", "bacteria", "animal", "Homo_sapiens", "marine", "other"]
def origin(simple_tax):
    if simple_tax in origins:
        return origins.index(simple_tax)


In [4]:
coconut["origin"] = coconut.simple_tax.map(origin)

In [5]:
def norm_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    smiles_n = Chem.MolToSmiles(mol, isomericSmiles=False)
    return smiles_n

coconut["norm_smiles"] = coconut["SMILES"].map(norm_smiles) 

In [6]:
coconut = coconut.query("origin != 6 and origin != 5 and origin != 4 and origin != 3").sort_values("origin", ascending=False)

In [7]:
coconut.replace(["NaN", 'NaT'], np.nan, inplace = True)
coconut = coconut.dropna()

/home/alice/miniconda3/envs/aipep/lib/python3.6/site-packages/pandas/core/missing.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/alice/miniconda3/envs/aipep/lib/python3.6/site-packages/pandas/core/missing.py:58: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [8]:
len(coconut)

60171

In [14]:
features = ['molecular_weight','number_of_carbons',\
       'number_of_nitrogens', 'number_of_oxygens',\
       'total_atom_number', 'bond_count', 'topoPSA', 'fcsp3', 'HBA',\
       'HBD', 'aLogP']
for feat in features:
    coconut[f"{feat}"]=coconut[f"{feat}"].map(float) 

In [15]:
if not os.path.exists(folder+"coconut_LSHforest_Micro_Plants"):
    fps = coconut["MAP4"].tolist()
    fps_ = []

    for fp in fps:
        fp_ = tm.VectorUint(fp)
        fps_.append(fp_)

    lf = tm.LSHForest(1024, 32)
    lf.batch_add(fps_)

    lf.index()
    lf.store(folder + 'coconut_LSHforest_Micro_Plants')
else:
    lf = tm.LSHForest(1024, 32)
    lf.restore(folder + 'coconut_LSHforest_Micro_Plants')

In [16]:
if not os.path.exists(folder+"coconut_tm_layout_Micro_Plants.pkl"):
    cfg = tm.LayoutConfiguration()

    # config
    cfg.node_size = 1 / 30
    cfg.mmm_repeats = 2
    cfg.sl_extra_scaling_steps = 5
    cfg.k = 20
    cfg.sl_scaling_type = tm.RelativeToAvgLength
    # Compute the layout

    x_, y_, s, t, gp = tm.layout_from_lsh_forest(lf, cfg)
    tm_layout = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}
    joblib.dump(tm_layout, folder+"coconut_tm_layout_Micro_Plants.pkl")
else:
    tm_layout = joblib.load(folder+"coconut_tm_layout_Micro_Plants.pkl")

In [17]:
tmp = coconut.copy()

idx = tmp[tmp["MW"] >= 1000].index
tmp.loc[idx, "MW"] = 1000

idx = tmp[tmp.HBA >= 20].index
tmp.loc[idx, "HBA"] = 20

idx = tmp[tmp.HBD >= 10].index
tmp.loc[idx, "HBD"] = 10

idx = tmp[tmp.aLogP >= 8].index
tmp.loc[idx, "aLogP"] = 8

idx = tmp[tmp.aLogP <= -2].index
tmp.loc[idx, "aLogP"] = -2

n_C = tmp.number_of_carbons.tolist()
n_O = tmp.number_of_oxygens.tolist()
n_N = tmp.number_of_nitrogens.tolist()
n_a = tmp.total_atom_number.tolist()
n_b = tmp.bond_count.tolist()
TPSA = tmp.topoPSA.tolist()
MW = tmp.MW.tolist()
fcsp3 = tmp.fcsp3.tolist()
HBA = tmp.HBA.tolist()
HBD = tmp.HBD.tolist()
alogp = tmp.aLogP.tolist()

isLipinski = tmp.isLipinski.tolist()

isPeptide = tmp.isPeptide.tolist()
hasSugar = tmp.hasSugar.tolist()
labels = tmp.TMAPlabel.tolist()
SMILES = tmp.SMILES.tolist()
ID = tmp.coconut_id.tolist()
origin = tmp.origin.tolist()

In [18]:
from matplotlib.colors import ListedColormap
from faerun import Faerun
custom_cmap = ListedColormap([ 'lightgray', "magenta"], name="custom")
custom_cmap2 = ListedColormap([ 'limegreen', "lightgray"], name="custom2")
custom_cmap3 = ListedColormap([ "lightgray","cyan", "magenta"], name="custom3")
custom_cmap4 = ListedColormap([ "limegreen", "blue", "tomato", "tomato", "pink", "cyan","lightgray"], name="custom4")
custom_cmap4_b = ListedColormap([ "limegreen", "cyan", "tomato", "tomato", "pink", "cyan","lightgray"], name="custom4")

groups = ["0-No", "1-Yes"]
labels_groups, groups = Faerun.create_categories(groups)

groups3 = ["0-plants", "1-fungi", "2-bacteria"]#, "3-animals", "4-Homo sapiens", "6-others"]
labels_groups3, groups3 = Faerun.create_categories(groups3)

In [19]:
faerun = Faerun(view="front", coords=False, title='MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants', clear_color="#ffffff",)
faerun.add_scatter("MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants",{"x": tm.VectorFloat(tm_layout["x"]), "y": tm.VectorFloat(tm_layout["y"]), "c": [MW, fcsp3, HBA,\
                                        HBD, alogp,  n_C, n_O, n_N, n_a, n_b, TPSA,\
                                           isPeptide, hasSugar, isLipinski, origin],\
                          "labels": labels}, has_legend=True, \
    colormap=["rainbow", "rainbow", "rainbow", "rainbow", "rainbow","rainbow","rainbow", "rainbow", "rainbow", "rainbow", "rainbow", \
             custom_cmap, custom_cmap, custom_cmap2,custom_cmap4], \
        point_scale=2, categorical=[False, False, False, False, False, False, False, False, False, False, False, True, True, True, True],\
        series_title=["MW", "Fraction sp3 C", "HBA count", "HBD count", "AlogP",'number of carbons',\
       'number of nitrogens', 'number of oxygens',\
       'total atom number', 'bond count', 'topoPSA',"Peptide", "Sugar", "is Lipinski", "Origin"], \
            max_legend_label=[">=1000", str(round(max(fcsp3),2)), ">=20",\
                              ">=10", ">=8", str(round(max(n_C),2)), str(round(max(n_O),2)),str(round(max(n_N),2)), \
                              str(round(max(n_a),2)), str(round(max(n_b),2)), str(round(max(TPSA),2)), None, None, None, None],\
                   min_legend_label=[str(round(min(MW),2)), str(round(min(fcsp3),2)), str(round(min(HBA),2)),\
                              str(round(min(HBD),2)), "<=2", str(round(min(n_C),2)), str(round(min(n_O),2)), str(round(min(n_N),2)), \
                              str(round(min(n_a),2)), str(round(min(n_b),2)), str(round(min(TPSA),2)), None, None, None, None],\
                    legend_labels=[None, None, None, None, None, None, None, None, None, None, None, labels_groups, labels_groups, labels_groups, labels_groups3])

faerun.add_tree("MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_tree",{"from": tm.VectorUint(tm_layout["s"]), "to": tm.VectorUint(tm_layout["t"])},\
                point_helper="MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants", color="aaaaaa")
faerun.plot('MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants', template = 'smiles')

/home/alice/Code/coconut_clean/MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants.html

In [20]:
faerun = Faerun(view="front", coords=False, title='MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b')
faerun.add_scatter("MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b",{"x": tm.VectorFloat(tm_layout["x"]), "y": tm.VectorFloat(tm_layout["y"]), "c": [MW, fcsp3, HBA,\
                                        HBD, alogp,  n_C, n_O, n_N, n_a, n_b, TPSA,\
                                           isPeptide, hasSugar, isLipinski, origin],\
                          "labels": labels}, has_legend=True, \
    colormap=["rainbow", "rainbow", "rainbow", "rainbow", "rainbow","rainbow","rainbow", "rainbow", "rainbow", "rainbow", "rainbow", \
             custom_cmap, custom_cmap, custom_cmap2,custom_cmap4_b], \
        point_scale=2, categorical=[False, False, False, False, False, False, False, False, False, False, False, True, True, True, True],\
        series_title=["MW", "Fraction sp3 C", "HBA count", "HBD count", "AlogP",'number of carbons',\
       'number of nitrogens', 'number of oxygens',\
       'total atom number', 'bond count', 'topoPSA',"Peptide", "Sugar", "is Lipinski", "Origin"], \
            max_legend_label=[">=1000", str(round(max(fcsp3),2)), ">=20",\
                              ">=10", ">=8", str(round(max(n_C),2)), str(round(max(n_O),2)),str(round(max(n_N),2)), \
                              str(round(max(n_a),2)), str(round(max(n_b),2)), str(round(max(TPSA),2)), None, None, None, None],\
                   min_legend_label=[str(round(min(MW),2)), str(round(min(fcsp3),2)), str(round(min(HBA),2)),\
                              str(round(min(HBD),2)), "<=2", str(round(min(n_C),2)), str(round(min(n_O),2)), str(round(min(n_N),2)), \
                              str(round(min(n_a),2)), str(round(min(n_b),2)), str(round(min(TPSA),2)), None, None, None, None],\
                    legend_labels=[None, None, None, None, None, None, None, None, None, None, None, labels_groups, labels_groups, labels_groups, labels_groups3])

faerun.add_tree("MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b_tree",{"from": tm.VectorUint(tm_layout["s"]), "to": tm.VectorUint(tm_layout["t"])},\
                point_helper="MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b", color="aaaaaa")
faerun.plot('MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b', template = 'smiles')

/home/alice/Code/coconut_clean/MAP4_curatedCOCONUT_TMAP_MicrobialAndPlants_b.html